In [1]:
#Import Libraries
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, precision_score, f1_score, classification_report
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn import metrics
import pandas as pd
from wordcloud import WordCloud
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import string
import itertools
import nltk
import re
import wandb
from wandb.keras import WandbCallback
import os
# Set a random seed for reproducibility
np.random.seed(42)

from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("wandb_api_key") 

wandb.login(key=my_secret)

2024-05-21 13:45:34.930806: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 13:45:34.930941: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 13:45:35.071483: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.ne

True

In [2]:
#Importing Datasets
data = pd.read_csv("/kaggle/input/abstracts/research-abstracts-labeled.csv")
Test_data = pd.read_csv("/kaggle/input/gpt-vs-human-a-corpus-of-research-abstracts/data_set.csv", low_memory=False)
#Checking if Dataset has any null Values
data.isnull().sum()

data = shuffle(data)
data = data.reset_index( drop=True )

#Removed duplicate datat
data.drop_duplicates(subset=['text'])
#Removed unnecessay columns
#data.drop(["title","word_count"],axis=1,inplace=True)
data.dropna()
data.describe()

data.head()

data.to_csv("sub.csv", index=False)

In [ ]:
columns_to_keep = ['title','abstract','is_ai_generated']
Test_data = Test_data[columns_to_keep]
Test_data.head()

#Test Data Cleaning
Test_data.isnull().sum()
Test_data = Test_data.dropna()

#data Cleaning
nltk.download('stopwords')
stop =  stopwords.words('english')

def remove_punctuation(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)
data['text'] = data['text'].apply(remove_punctuation)
data['text'] = data['text'].str.replace('\n','')
data['text'] = data['text'].str.replace('\d+', '', regex=True)
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data['text'] = data['text'].str.lower()
data.head()

In [ ]:
#Roberta Model
import random
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from transformers import RobertaTokenizer, RobertaModel
import tensorflow_text as text

tokenizer = RobertaTokenizer.from_pretrained("/kaggle/input/roberta-base")

def tokenize_and_truncate(text, max_length=512):
    if isinstance(text, float):
        text = str(int(text))  # Convert float to integer and then to string
    # Tokenize and truncate
    tokens = tokenizer.tokenize(text)[:max_length]  # leaving space for special tokens
    # Add the special tokens
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    
    return token_ids

#Adding Tokens to both Datasets
data["tokens"] = data["text"].apply(lambda x: tokenize_and_truncate(x))
Test_data["tokens"] = Test_data["abstract"].apply(lambda x: tokenize_and_truncate(x))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class RobertaForBinaryClassification(nn.Module):
    def __init__(self):
        super(RobertaForBinaryClassification, self).__init__()

        self.roberta = RobertaModel.from_pretrained("/kaggle/input/roberta-base")
        hidden_size = self.roberta.config.hidden_size

        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.fc2 = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask=None):
        outputs = self.roberta(input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]

        x = self.fc1(cls_output)
        x = self.relu(x)
        x = self.dropout(x)
        logits = self.fc2(x)
        probs = self.sigmoid(logits)
        return probs
    
class TextDataset(Dataset):
    def __init__(self, token_lists, labels, max_token_len=512):
        self.token_lists = token_lists
        self.labels = labels
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.token_lists)

    def __getitem__(self, idx):
        # Pad the token lists to the max length
        token_list = self.token_lists[idx][:self.max_token_len]  # truncate if longer than max length
        padded_tokens = token_list + [0] * (self.max_token_len - len(token_list))  # pad with zeros
        attention_mask = [1 if i < len(token_list) else 0 for i in range(self.max_token_len)]
        return torch.tensor(padded_tokens), torch.tensor(attention_mask), torch.tensor(self.labels[idx])

In [ ]:
CFG = dict(
    optimiser = 'Adam',
    learning_rate = 1e-6,
)

epoch = [1,2,3,4,5]

In [ ]:
#Splitting the dataframe to have train and testing data
train_data, test_data = train_test_split(data, test_size=0.3, random_state = 42)

In [ ]:
from sklearn.metrics import recall_score
from wandb.keras import WandbCallback
from tqdm.auto import tqdm
# Create Dataset and DataLoader

#Training
train_dataset = TextDataset(train_data["tokens"].tolist(), train_data["label"].tolist())
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#Tetsing
test_dataset = TextDataset(test_data["tokens"].tolist(), test_data["label"].tolist())
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Instantiate the model
model = RobertaForBinaryClassification()
model = nn.DataParallel(model)
model.to(device)
# Define loss function and optimizer
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=CFG['learning_rate'])

In [ ]:
# Lists to store metrics for each epoch
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

#Training
from sklearn.metrics import recall_score
from wandb.keras import WandbCallback
from tqdm.auto import tqdm
for x in epoch:
    #Initialise Run
    run = wandb.init(name = "ROBERTA_Hyperparameter",
                    project = "Dissertation_FinalResults",
                    config = CFG,)
    model.train()
    train_loss = 0
    total_correct = 0
    total_samples = 0

    # Initialize lists to store true labels and predictions
    all_predictions = []
    all_labels = []

    for tokens, attention_masks, labels in tqdm(train_loader):
        tokens, attention_masks, labels = tokens.to(device), attention_masks.to(device), labels.to(device)  # Move to GPU if available
        optimizer.zero_grad()
        labels = labels.float().unsqueeze(1)  # Adjust labels' shape if necessary

        outputs = model(tokens, attention_mask=attention_masks)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        # Run inference
        outputs = model(tokens, attention_mask=attention_masks)
        # Convert outputs to predictions (binary classification)
        predictions = (outputs > 0.5).int() 
        total_correct += (predictions == labels).sum().item()
        total_samples += labels.size(0)
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    
     # Calculate average training loss and accuracy for the epoch
    avg_train_loss = train_loss / len(train_loader)
    train_accuracy = total_correct / total_samples
    
    
    # Flatten the lists and convert to numpy arrays
    all_predictions = np.array(all_predictions).flatten()
    all_labels = np.array(all_labels).flatten()

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions)
    recall = recall_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions)
    
     # Validation Phase
    model.eval()
    val_loss = 0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for tokens, attention_masks, labels in tqdm(test_loader):
            tokens, attention_masks, labels = tokens.to(device), attention_masks.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(tokens, attention_mask=attention_masks)
            loss = loss_function(outputs, labels)
            val_loss += loss.item()

            # Compute accuracy
            predictions = (outputs > 0.5).int()
            total_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)

    # Calculate average validation loss and accuracy for the epoch
    avg_val_loss = val_loss / len(test_loader)
    val_accuracy = total_correct / total_samples

    # Store metrics for plotting and logging
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy) 
    
        
    wandb.log({
            "Epoch":x,
            "Accuracy": accuracy,
            "Precision": precision,
            "F1 Score": f1,
            "Recall": recall,
            "Train Loss": avg_train_loss,
            "Train Accuracy": train_accuracy,
            "Val Loss": avg_val_loss,
            "Val Accuracy": val_accuracy
        })
    # Complete W&B run
    run.finish()     

In [ ]:
Train_Labels = pd.DataFrame({
    'Original Labels' : all_labels,
    'Predicted Labels': all_predictions
})
Train_Labels

In [ ]:
Train_Labels.to_csv("submission.csv", index=False)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm = confusion_matrix(all_labels,all_predictions)

sns.heatmap(cm, 
            annot=True,
            fmt='g', 
            xticklabels=['Machine','Human'],
            yticklabels=['Machine','Human'])
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.title('Confusion Matrix',fontsize=17)
plt.show()